## Generate the path

In [ ]:
import sys
import os
import numpy as np
import pandas as pd


In [ ]:
# Add project root to path
sys.path.append('..') # Add '#' at the beginning of this line if the pipeline does not work on your computer


## load data

In [2]:
from src.data_loader import DataLoader

# Load all data
loader = DataLoader()
data = loader.load_data()

price_data = data['price_data']
news_data = data['news_data']
fundamental_data = data['fundamental_data']

print("SUCCESS: All data loaded!")
print(f"Price data: {price_data.shape}")
print(f"News data: {news_data.shape}")
print(f"Fundamental data: {fundamental_data.shape}")

print("Price data columns:", price_data.columns.tolist()[:5], "...")
print("News data columns:", news_data.columns.tolist()[:5], "...")
print("Fundamental data columns:", fundamental_data.columns.tolist()[:5], "...")

print("Fundamental data sample:")
print(fundamental_data.head(3))

Loading config from: f:\Upenn\CIS5450 - big data analysis\project_repo\CIS545_Repo\config\parameters.yaml
Loading data...
Loading price data from: f:\Upenn\CIS5450 - big data analysis\project_repo\CIS545_Repo\data/raw/AAPL.US_M5.xlsx
Price data shape: (104028, 6)
Loading news data from: f:\Upenn\CIS5450 - big data analysis\project_repo\CIS545_Repo\data/raw/apple_news_data.xlsx
News data shape: (42127, 27)
Loading fundamental data from: f:\Upenn\CIS5450 - big data analysis\project_repo\CIS545_Repo\data/raw/iw0erafaqkjcbrw6.xlsx
Fundamental data shape: (76, 78)
SUCCESS: All data loaded!
Price data: (104028, 6)
News data: (42127, 27)
Fundamental data: (76, 78)
Price data columns: ['datetime', 'open', 'high', 'low', 'close'] ...
News data columns: ['date', 'title', 'content', 'link', 'symbols'] ...
Fundamental data columns: ['gvkey', 'permno', 'adate', 'qdate', 'public_date'] ...
Fundamental data sample:
   gvkey  permno      adate      qdate public_date   CAPEI     bm     evm  \
0   1690 

In [ ]:
# Check data structure before merging
print(" CHECKING DATA STRUCTURE")

print("\n" + "="*50)
print(" PRICE DATA STRUCTURE")
print("="*50)
print(f"Shape: {price_data.shape}")
print(f"Columns: {price_data.columns.tolist()}")
print(f"Data types:")
print(price_data.dtypes)
print(f"\nFirst 2 rows:")
print(price_data.head(2))

print("\n" + "="*50)
print(" NEWS DATA STRUCTURE")
print("="*50)
print(f"Shape: {news_data.shape}")
print(f"Columns: {news_data.columns.tolist()}")
print(f"Data types:")
print(news_data.dtypes)
print(f"\nFirst 2 rows:")
print(news_data.head(2))

print("\n" + "="*50)
print(" FUNDAMENTAL DATA STRUCTURE")
print("="*50)
print(f"Shape: {fundamental_data.shape}")
print(f"Columns: {fundamental_data.columns.tolist()}")
print(f"Data types:")
print(fundamental_data.dtypes)
print(f"\nFirst 2 rows:")
print(fundamental_data.head(2))

# Check for sentiment-related columns in news data
print("\n" + "="*50)
print(" SENTIMENT COLUMNS IN NEWS DATA")
print("="*50)
sentiment_cols = [col for col in news_data.columns if 'sentiment' in col.lower()]
print(f"Sentiment-related columns: {sentiment_cols}")
if sentiment_cols:
    for col in sentiment_cols:
        print(f"  {col}: {news_data[col].dtype}")
        print(f"  Sample values: {news_data[col].head(3).tolist()}")

# Check date columns in each dataset
print("\n" + "="*50)
print(" DATE COLUMNS IN EACH DATASET1")
print("="*50)
print("Price data date columns:", [col for col in price_data.columns if 'date' in col.lower() or 'time' in col.lower()])
print("News data date columns:", [col for col in news_data.columns if 'date' in col.lower() or 'time' in col.lower()])
print("Fundamental data date columns:", [col for col in fundamental_data.columns if 'date' in col.lower() or 'time' in col.lower()])

🔍 CHECKING DATA STRUCTURE

📊 PRICE DATA STRUCTURE
Shape: (104028, 6)
Columns: ['datetime', 'open', 'high', 'low', 'close', 'volume']
Data types:
datetime    datetime64[ns]
open               float64
high               float64
low                float64
close              float64
volume               int64
dtype: object

First 2 rows:
             datetime   open   high    low  close  volume
0 2018-11-30 17:50:00  44.60  44.64  44.55  44.63  179494
1 2018-11-30 17:55:00  44.63  44.63  44.52  44.52  218881

📰 NEWS DATA STRUCTURE
Shape: (42127, 27)
Columns: ['date', 'title', 'content', 'link', 'symbols', 'tags', 'sentiment_polarity', 'sentiment_neg', 'sentiment_neu', 'sentiment_pos', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26']
Data types:
date                   object
title        

## Merge

### 1: Check data structure in all datasets

In [7]:
print("Price Data:")
print(f"  Shape: {price_data.shape}")
print(f"  Columns: {price_data.columns.tolist()}")
print(f"  First datetime: {price_data['datetime'].min()}")
print(f"  Last datetime: {price_data['datetime'].max()}")

print("\nNews Data:")
print(f"  Shape: {news_data.shape}")
print(f"  Date columns: {[col for col in news_data.columns if 'date' in col.lower()]}")
print(f"  Sentiment columns: {[col for col in news_data.columns if 'sentiment' in col.lower()]}")

print("\nFundamental Data:")
print(f"  Shape: {fundamental_data.shape}")
print(f"  Date columns: {[col for col in fundamental_data.columns if 'date' in col.lower()]}")
print(f"  Key metrics: {['CAPEI', 'bm', 'evm', 'pe_op_basic', 'roa', 'roe']}")

Price Data:
  Shape: (104028, 7)
  Columns: ['datetime', 'open', 'high', 'low', 'close', 'volume', 'merge_date']
  First datetime: 2018-11-30 17:50:00
  Last datetime: 2024-03-13 23:00:00

News Data:
  Shape: (29751, 28)
  Date columns: ['date', 'merge_date']
  Sentiment columns: ['sentiment_polarity', 'sentiment_neg', 'sentiment_neu', 'sentiment_pos']

Fundamental Data:
  Shape: (76, 79)
  Date columns: ['adate', 'qdate', 'public_date', 'merge_date']
  Key metrics: ['CAPEI', 'bm', 'evm', 'pe_op_basic', 'roa', 'roe']


### 2: Create unified date columns for merging

In [8]:
# Price data: extract date from datetime
price_data['merge_date'] = pd.to_datetime(price_data['datetime']).dt.date
print(f" Price data: created merge_date from datetime")
print(f"   Date range: {price_data['merge_date'].min()} to {price_data['merge_date'].max()}")

# News data: convert and clean date column
news_data['merge_date'] = pd.to_datetime(news_data['date'], errors='coerce').dt.date
initial_news_count = len(news_data)
news_data = news_data.dropna(subset=['merge_date'])
cleaned_news_count = len(news_data)
print(f" News data: created merge_date from date column")
print(f"   Removed {initial_news_count - cleaned_news_count} rows with invalid dates")
print(f"   Date range: {news_data['merge_date'].min()} to {news_data['merge_date'].max()}")

# Fundamental data: use public_date as reference date
fundamental_data['merge_date'] = pd.to_datetime(fundamental_data['public_date']).dt.date
print(f" Fundamental data: created merge_date from public_date")
print(f"   Date range: {fundamental_data['merge_date'].min()} to {fundamental_data['merge_date'].max()}")

 Price data: created merge_date from datetime
   Date range: 2018-11-30 to 2024-03-13
 News data: created merge_date from date column
   Removed 0 rows with invalid dates
   Date range: 2016-02-19 to 2024-11-27
 Fundamental data: created merge_date from public_date
   Date range: 2018-01-31 to 2024-04-30


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_27400\2405703928.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_data['merge_date'] = pd.to_datetime(news_data['date'], errors='coerce').dt.date


### 3: Process sentiment data and convert to numeric

In [10]:
sentiment_cols = ['sentiment_polarity', 'sentiment_neg', 'sentiment_neu', 'sentiment_pos']
converted_cols = []

for col in sentiment_cols:
    if col in news_data.columns:
        news_data[col] = pd.to_numeric(news_data[col], errors='coerce')
        converted_cols.append(col)
        valid_count = news_data[col].notna().sum()
        print(f"Converted {col}: {valid_count} valid values")

print(f"   Total sentiment columns processed: {len(converted_cols)}")

Converted sentiment_polarity: 29462 valid values
Converted sentiment_neg: 29474 valid values
Converted sentiment_neu: 29496 valid values
Converted sentiment_pos: 29476 valid values
   Total sentiment columns processed: 4


### 4: Aggregate sentiment data by date

In [11]:
daily_sentiment = news_data.groupby('merge_date').agg({
    'sentiment_polarity': 'mean',
    'sentiment_neg': 'mean', 
    'sentiment_neu': 'mean',
    'sentiment_pos': 'mean',
    'title': 'count'
}).reset_index()

# Rename columns for clarity
daily_sentiment.columns = [
    'merge_date', 
    'avg_sentiment_polarity', 
    'avg_sentiment_neg', 
    'avg_sentiment_neu', 
    'avg_sentiment_pos', 
    'daily_news_count'
]

print(f"   Created daily sentiment aggregation")
print(f"   Unique dates with news: {len(daily_sentiment)}")
print(f"   Total news articles: {daily_sentiment['daily_news_count'].sum()}")
print(f"   Aggregated columns: {daily_sentiment.columns.tolist()}")

   Created daily sentiment aggregation
   Unique dates with news: 1574
   Total news articles: 29751
   Aggregated columns: ['merge_date', 'avg_sentiment_polarity', 'avg_sentiment_neg', 'avg_sentiment_neu', 'avg_sentiment_pos', 'daily_news_count']


### 5: Merge price data with daily sentiment

In [12]:
price_with_sentiment = pd.merge(
    price_data, 
    daily_sentiment, 
    on='merge_date', 
    how='left'  # Keep all price records
)

print(f"   Successfully merged price and sentiment data")
print(f"   Before merge: {price_data.shape}")
print(f"   After merge: {price_with_sentiment.shape}")

# Check merge results
sentiment_coverage = price_with_sentiment['avg_sentiment_polarity'].notna().sum()
total_records = len(price_with_sentiment)
print(f"   Records with sentiment data: {sentiment_coverage}/{total_records} ({sentiment_coverage/total_records*100:.1f}%)")

   Successfully merged price and sentiment data
   Before merge: (104028, 7)
   After merge: (104028, 12)
   Records with sentiment data: 74220/104028 (71.3%)


### 6: Prepare fundamental data for merging

In [ ]:
# Select key fundamental metrics
key_fundamental_metrics = [
    'merge_date', 'CAPEI', 'bm', 'evm', 'pe_op_basic', 'pe_op_dil', 
    'roa', 'roe', 'npm', 'ps', 'de_ratio'
]

# Filter to only include columns that exist
available_metrics = [col for col in key_fundamental_metrics if col in fundamental_data.columns]
fundamental_subset = fundamental_data[available_metrics].copy()

print(f" Selected fundamental metrics:")
for metric in available_metrics:
    if metric != 'merge_date':
        valid_count = fundamental_subset[metric].notna().sum()
        print(f"   - {metric}: {valid_count} valid values")


 STEP 6: PREPARING FUNDAMENTAL DATA
 Selected fundamental metrics:
   - CAPEI: 76 valid values
   - bm: 76 valid values
   - evm: 76 valid values
   - pe_op_basic: 76 valid values
   - pe_op_dil: 76 valid values
   - roa: 76 valid values
   - roe: 76 valid values
   - npm: 76 valid values
   - ps: 76 valid values
   - de_ratio: 76 valid values


### 7: Final merge with fundamental data

In [14]:

merged_data = pd.merge(
    price_with_sentiment,
    fundamental_subset,
    on='merge_date',
    how='left'  # Keep all price-sentiment records
)

print(f"   Final merge completed!")
print(f"   Final dataset shape: {merged_data.shape}")
print(f"   Total columns: {len(merged_data.columns)}")

# Check fundamental data coverage
if 'pe_op_basic' in merged_data.columns:
    fundamental_coverage = merged_data['pe_op_basic'].notna().sum()
    print(f"   Records with fundamental data: {fundamental_coverage}/{len(merged_data)} ({fundamental_coverage/len(merged_data)*100:.1f}%)")

   Final merge completed!
   Final dataset shape: (104028, 22)
   Total columns: 22
   Records with fundamental data: 3595/104028 (3.5%)


### 8: Display final merged dataset summary

In [19]:
print("="*60)
print("FINAL MERGED DATASET OVERVIEW")
print("="*60)

print(f" Dataset Shape: {merged_data.shape}")
print(f" Date Range: {merged_data['merge_date'].min()} to {merged_data['merge_date'].max()}")
print(f" Time Frequency: 5-minute intervals")

print(f" COLUMN CATEGORIES:")
price_cols = [col for col in merged_data.columns if col in ['datetime', 'open', 'high', 'low', 'close', 'volume', 'merge_date']]
sentiment_cols = [col for col in merged_data.columns if 'sentiment' in col.lower() or 'news' in col.lower()]
fundamental_cols = [col for col in merged_data.columns if col in available_metrics and col != 'merge_date']

print(f"   Price Data: {len(price_cols)} columns")
print(f"   Sentiment Data: {len(sentiment_cols)} columns") 
print(f"   Fundamental Data: {len(fundamental_cols)} columns")

print(f"   DATA COMPLETENESS:")
print(f"   Total Records: {len(merged_data):,}")
print(f"   Unique Dates: {merged_data['merge_date'].nunique()}")
print(f"   Records with Sentiment: {merged_data['avg_sentiment_polarity'].notna().sum():,}")
print(f"   Records with Fundamentals: {merged_data[fundamental_cols[0]].notna().sum() if fundamental_cols else 0:,}")

print(f"   SAMPLE DATA (First 3 rows):")
sample_cols = ['datetime', 'close', 'volume', 'daily_news_count', 'avg_sentiment_polarity']
if fundamental_cols:
    sample_cols.append(fundamental_cols[0])
    
print(merged_data[sample_cols].head(3))

FINAL MERGED DATASET OVERVIEW
 Dataset Shape: (104028, 22)
 Date Range: 2018-11-30 to 2024-03-13
 Time Frequency: 5-minute intervals
 COLUMN CATEGORIES:
   Price Data: 7 columns
   Sentiment Data: 5 columns
   Fundamental Data: 10 columns
   DATA COMPLETENESS:
   Total Records: 104,028
   Unique Dates: 1376
   Records with Sentiment: 74,220
   Records with Fundamentals: 3,595
   SAMPLE DATA (First 3 rows):
             datetime  close  volume  daily_news_count  \
0 2018-11-30 17:50:00  44.63  179494               1.0   
1 2018-11-30 17:55:00  44.52  218881               1.0   
2 2018-11-30 18:00:00  44.49  364114               1.0   

   avg_sentiment_polarity   CAPEI  
0                   0.998  17.191  
1                   0.998  17.191  
2                   0.998  17.191  


### 9: Save the merged dataset

In [17]:

output_path = "../data/processed/merged_data.pkl"
merged_data.to_pickle(output_path)

print(f" Merged data saved to: {output_path}")
print(f" File size: {os.path.getsize(output_path) / 1024 / 1024:.2f} MB")

print("  DATA MERGING PROCESS COMPLETED SUCCESSFULLY!")
print("="*60)

 Merged data saved to: ../data/processed/merged_data.pkl
 File size: 17.96 MB
  DATA MERGING PROCESS COMPLETED SUCCESSFULLY!


In [20]:
# Quick inspection of the saved file
import pandas as pd
data = pd.read_pickle("../data/processed/merged_data.pkl")
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104028 entries, 0 to 104027
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   datetime                104028 non-null  datetime64[ns]
 1   open                    104028 non-null  float64       
 2   high                    104028 non-null  float64       
 3   low                     104028 non-null  float64       
 4   close                   104028 non-null  float64       
 5   volume                  104028 non-null  int64         
 6   merge_date              104028 non-null  object        
 7   avg_sentiment_polarity  74220 non-null   float64       
 8   avg_sentiment_neg       74220 non-null   float64       
 9   avg_sentiment_neu       74220 non-null   float64       
 10  avg_sentiment_pos       74220 non-null   float64       
 11  daily_news_count        74220 non-null   float64       
 12  CAPEI                   3595 n